看看均线策略。

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

我们先指定一个特定的基金，特定的时间段来分析吧。

In [ ]:
# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

首先看看这个基金的基准表现，就是在开始时间就直接买入，然后一直持有，看具体的收益率。

In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='baseline',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

那么策略基准线大概就是这样了，7年多的时间1.8倍。

In [ ]:
ret = trdb2py.getAssetCandles2(trdb2cfg, asset, tsStart, tsEnd, indicators=['sma.29', 'sma.60', 'sma.120', 'sma.240'])
# print(ret)
# ret
trdb2py.showAssetCandles2(asset, ret, indicators=['sma.29', 'sma.60', 'sma.120', 'sma.240'], toImg=isStaticImg, width=width, height=height)

接下来，测一下均线

In [ ]:
lstparams = []

for ema in range(2, 61):    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='ema{}'.format(ema),
    ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

可以看到，简单的均线策略，最好的情况也就是将上涨的趋势把握住，跳过下跌大的区间，当然，基本上上涨和下跌都会有损失。

这里粒度非常重要，看估计很难看出来，只能试。

In [ ]:
lstparams = []

for ema0 in range(2, 61):
    for ema1 in range(2, 61):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['upcross'],
            strVals=['ema.{}'.format(ema0)],
        )

        buy1 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
        )
        
        buy2 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['up'],
            strVals=['ema.{}'.format(ema1)],
        )
        
        buy3 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
        )        

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['downcross'],
            strVals=['ema.{}'.format(ema0)],
        )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0, buy1, buy2, buy3])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell) 
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='ema{}&ema{}'.format(ema0, ema1),
        ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=3)

# trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
asset = 'jqdata.000300_XSHG|1d'
# asset = 'jqdata.000905_XSHG'
# asset = 'jqdata.000932_XSHG'

lstparams = []

ema0 = 30
ema1 = 51

buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['upcross'],
    strVals=['ema.{}'.format(ema0)],
)

buy1 = trdb2py.trading2_pb2.CtrlCondition(
    name='waittostart',
    vals=[ema0],
)

buy2 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.{}'.format(ema1)],
)

buy3 = trdb2py.trading2_pb2.CtrlCondition(
    name='waittostart',
    vals=[ema1],
)        

sell0 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['downcross'],
    strVals=['ema.{}'.format(ema0)],
)

s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),
)

s0.buy.extend([buy0, buy1, buy2, buy3])
s0.sell.extend([sell0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsSell.CopyFrom(paramssell) 
s0.paramsInit.CopyFrom(paramsinit)        
lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='ema{}&ema{}'.format(ema0, ema1),
))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2)

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

这里策略复杂了一些，考虑了2条均线，可以明显看出，最好的策略相比单均线，有了明显的提升。

In [ ]:
asset = 'jrj.510310'

lstparams = []

for ema0 in range(2, 61):
    for ema1 in range(ema0, 61):
        for ema2 in range(ema1, 61):
            buy0 = trdb2py.trading2_pb2.CtrlCondition(
                name='indicatorsp',
                operators=['upcross'],
                strVals=['ema.{}'.format(ema0)],
            )

            buy1 = trdb2py.trading2_pb2.CtrlCondition(
                name='waittostart',
                vals=[ema0],
            )

            buy2 = trdb2py.trading2_pb2.CtrlCondition(
                name='indicatorsp',
                operators=['up'],
                strVals=['ema.{}'.format(ema1)],
            )

            buy3 = trdb2py.trading2_pb2.CtrlCondition(
                name='waittostart',
                vals=[ema1],
            )        

            sell0 = trdb2py.trading2_pb2.CtrlCondition(
                name='indicatorsp',
                operators=['downcross'],
                strVals=['ema.{}'.format(ema2)],
            )

            s0 = trdb2py.trading2_pb2.Strategy(
                name="normal",
                asset=trdb2py.str2asset(asset),
            )

            s0.buy.extend([buy0, buy1, buy2, buy3])
            s0.sell.extend([sell0])
            s0.paramsBuy.CopyFrom(paramsbuy)
            s0.paramsSell.CopyFrom(paramssell) 
            s0.paramsInit.CopyFrom(paramsinit)        
            lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
                assets=[trdb2py.str2asset(asset)],
                startTs=tsStart,
                endTs=tsEnd,
                strategies=[s0],
                title='ema{}&ema{}&ema{}'.format(ema0, ema1, ema2),
            ))
 
# lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=3.2)

# trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

dfpnl1 = dfpnl[dfpnl['totalReturns'] > 3]

# dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
# asset = 'jqdata.000300_XSHG'
asset = 'jqdata.000905_XSHG|1d'
# asset = 'jqdata.000932_XSHG|1d'

lstparams = []

ema0 = 5
ema1 = 9
ema2 = 29

buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['upcross'],
    strVals=['ema.{}'.format(ema0)],
)

buy1 = trdb2py.trading2_pb2.CtrlCondition(
    name='waittostart',
    vals=[ema0],
)

buy2 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.{}'.format(ema1)],
)

buy3 = trdb2py.trading2_pb2.CtrlCondition(
    name='waittostart',
    vals=[ema1],
)        

sell0 = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['downcross'],
    strVals=['ema.{}'.format(ema2)],
)

s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),
)

s0.buy.extend([buy0, buy1, buy2, buy3])
s0.sell.extend([sell0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsSell.CopyFrom(paramssell) 
s0.paramsInit.CopyFrom(paramsinit)        
lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='ema{}&ema{}&ema{}'.format(ema0, ema1, ema2),
))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=3)

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

我们把策略继续做复杂，就会发现还能有提升，但切换到回测数据以外的数据，会发现其实还不如上次的

这个也很好理解，我们拿一套数据，不断的细分拟合，不断加限制条件，条件越多，肯定越容易得到一个好的盈利曲线

但这种很容易就过拟合，并不一定适用更多的情况

特别对均线来说，如果粒度太小，是很容易过拟合的

对沪深300来说，基本上只要能经过2个牛市，就至少是10倍以上的回报

第一次积累筹码，第二次很容易就能得到一个很大的提升

均线策略不太能跨品种，也就是说沪深300的，不能简单的切到上证500来用

但波动越大，肯定收益越高

In [ ]:
asset = 'jrj.510310'

lstparams = []

for ema0 in range(2, 61):
    for ema1 in range(2, 61):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatordp',
            operators=['upcross'],
            strVals=['ema.{}'.format(ema0), 'ema.{}'.format(ema1)],
        )

        buy1 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
        )
        
        buy2 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
        )        

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatordp',
            operators=['downcross'],
            strVals=['ema.{}'.format(ema0), 'ema.{}'.format(ema1)],
        )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0, buy1, buy2])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell) 
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='ema{}&ema{}'.format(ema0, ema1),
        ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=20)

# trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

dfpnl1 = dfpnl[dfpnl['totalReturns'] > 3]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)